# Intergrated Gradients of the nl-BERT-large Model Prior to Fine-tuning
In this notebook we:

1. Employ the model without any fine-tuning
2. extract Intergrated Gradients (IG) values for the model to note which features (here tokens) contributes to the classification of either "female" or "male" artists in the pre-fine-tuning state of the nl-BERT-large model.

The output os this can then be compared to those of the model post fine-tuning in the `IG_DIFFERENTIALS.ipynb`.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#### Install packages:
!pip install -q transformers transformers-interpret datasets evaluate tensorflow spacy spacy_langdetect shap matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 925.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 998.1/998.1 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00


In [5]:
### Importing packages:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, AutoModelForPreTraining, Trainer, TrainingArguments, AutoModelForSequenceClassification, BertTokenizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import EarlyStoppingCallback
import matplotlib.pyplot as plt

Loading the pretrained model:

In [ ]:
from transformers import BertConfig

# the pretrained model:
original_model = "NbAiLab/nb-bert-large"

model = AutoModelForSequenceClassification.from_pretrained(original_model)

# Load the tokenizer from the original pre-trained model
tokenizer = AutoTokenizer.from_pretrained(original_model)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/NLP_EX_COLAB/eval_outputs/NBL_df_classification_report.csv')
texts = df['Text'].tolist() # extracting all texts to a list

In [ ]:
from transformers_interpret import SequenceClassificationExplainer

# Create a sequence classification explainer
explainer = SequenceClassificationExplainer(
    model=model,
    tokenizer=tokenizer
)


### Extracting and saving the IG values 5 batches at a time
This is done due to limited GPU capacity. Everything beyond 5 batches, crashes the run.
Thus, the explainer function is used continously at each 5 batch and the output saved for each round.

In [ ]:
import os
import json

# Function to divide texts into batches
def batchify(texts, batch_size):
    return [texts[i:i + batch_size] for i in range(0, len(texts), batch_size)]

# Define batch size and divide texts into batches
batch_size = 5
text_batches = batchify(texts, batch_size)

# File to track processed batches
processed_batches_file = '/content/drive/MyDrive/Colab_Notebooks/NLP_EX_COLAB/IG/pretrained_IG_output/processed_batches.txt'

# Function to save IG values
def save_shap_values(batch_index, shap_values):
    with open(f'/content/drive/MyDrive/Colab_Notebooks/NLP_EX_COLAB/IG/pretrained_IG_output/shap_values_batch_{batch_index}.json', 'w') as f:
        json.dump(shap_values, f)

# Function to get the last processed batch index
def get_last_processed_batch():
    if os.path.exists(processed_batches_file):
        with open(processed_batches_file, 'r') as f:
            return int(f.read().strip())
    return 0

In [ ]:
# Start processing from the next batch after the last processed batch
last_processed_batch = get_last_processed_batch()

for batch_index, batch in enumerate(text_batches):
    if batch_index <= last_processed_batch:
        continue

    # Process batch
    batch_shap_values = [explainer(text) for text in batch]

    # Save IG values
    save_shap_values(batch_index, batch_shap_values)

    # Update processed batches file
    with open(processed_batches_file, 'w') as f:
        f.write(str(batch_index))

    # Clear memory if needed
    del batch_shap_values
    torch.cuda.empty_cache()

In [ ]:
#### GET BATCH 0
text0 = df['Text'][0:5].tolist()
batch_shap_values = [explainer(text) for text in text0]
save_shap_values(0, batch_shap_values)